In [ ]:
!git clone https://github.com/YIKUAN8/Transformers-VQA.git
%cd Transformers-VQA/
!pip install -r requirements.txt

Cloning into 'Transformers-VQA'...
remote: Enumerating objects: 182, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 182 (delta 67), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (182/182), 1.94 MiB | 4.82 MiB/s, done.
Resolving deltas: 100% (67/67), done.
/content/Transformers-VQA
     |████████████████████████████████| 132 kB 12.5 MB/s 
     |████████████████████████████████| 8.6 MB 39.2 MB/s 
     |████████████████████████████████| 79 kB 6.0 MB/s 
     |████████████████████████████████| 138 kB 33.1 MB/s 
     |████████████████████████████████| 127 kB 39.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 

In [ ]:
import pickle
from param import args
import pandas as pd

In [ ]:
#In param.py code, replace 'args = parser.parse_args()' with 'args = parser.parse_args([])' 

In [ ]:
!wget --no-check-certificate https://nlp1.cs.unc.edu/data/model_LXRT.pth -P models/pretrained/

--2022-03-25 13:47:23--  https://docs.google.com/uc?export=download&confirm=t&id=1kuPr187zWxSJbtCbVW87XzInXltM-i9Y
Resolving docs.google.com (docs.google.com)... 66.102.1.138, 66.102.1.101, 66.102.1.102, ...
Connecting to docs.google.com (docs.google.com)|66.102.1.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-5c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/st0rcief55jqjkm5lr52inepomrl38qs/1648215975000/12682199137908966719/*/1kuPr187zWxSJbtCbVW87XzInXltM-i9Y?e=download [following]
--2022-03-25 13:47:23--  https://doc-0g-5c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/st0rcief55jqjkm5lr52inepomrl38qs/1648215975000/12682199137908966719/*/1kuPr187zWxSJbtCbVW87XzInXltM-i9Y?e=download
Resolving doc-0g-5c-docs.googleusercontent.com (doc-0g-5c-docs.googleusercontent.com)... 142.251.5.132, 2a00:1450:400c:c1b::84
Connecting to doc-0g-5c-docs.googleusercontent.com (doc-0g-5c-docs.goo

In [ ]:
args.__dict__

{'batch_size': 32,
 'dropout': 0.1,
 'epochs': 2,
 'fast': False,
 'from_scratch': False,
 'load_pretrained': None,
 'load_trained': None,
 'lr': 0.0001,
 'max_seq_length': 20,
 'mce_loss': False,
 'model': 'lxmert',
 'multiGPU': False,
 'num_workers': 0,
 'optim': 'bert',
 'optimizer': 'bert',
 'output': 'models/trained/',
 'seed': 9595,
 'test': None,
 'tiny': False,
 'tqdm': True,
 'train': 'train,nominival',
 'valid': 'minival'}

In [ ]:
args.batch_size = 18
args.epochs = 2
args.model = 'lxmert' # use lxmert
args.load_pretrained = 'models/pretrained/model_LXRT.pth' #load pretrained visualbert model
args.max_seq_length = 128 #truncate or pad report lengths to 128 subwords

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import numpy as np
class PatientDataset(Dataset):
  def __init__(self, df, vf, model = 'visualbert'):
    self.visual_features = vf
    self.id_list = df.study_id.tolist()
    self.report_list = df.RadiologyReport.tolist()
    self.findings_list = df.columns[5:-2]
    self.target_list = df[self.findings_list].to_numpy().astype(np.float32)
    self.model = model

  def __len__(self):
    return len(self.id_list)

  def __getitem__(self, item):
    cxr_id = self.id_list[item]
    target = self.target_list[item]
    feats, boxes, img_w, img_h = self.visual_features[cxr_id]
    report = self.report_list[item]
    if self.model == 'uniter':
      boxes = self._uniterBoxes(boxes)
    if self.model == 'lxmert':
      boxes[:, (0, 2)] /= img_w
      boxes[:, (1, 3)] /= img_h
    #print(feats.shape, boxes.shape)
    return cxr_id, feats, boxes, report, target

  def _uniterBoxes(self, boxes):#uniter requires a 7-dimensiom beside the regular 4-d bbox
    new_boxes = np.zeros((boxes.shape[0],7),dtype='float32')
    new_boxes = np.zeros((boxes.shape[0],7),dtype='float32')
    new_boxes[:,1] = boxes[:,0]
    new_boxes[:,0] = boxes[:,1]
    new_boxes[:,3] = boxes[:,2]
    new_boxes[:,2] = boxes[:,3]
    new_boxes[:,4] = new_boxes[:,3]-new_boxes[:,1] #w
    new_boxes[:,5] = new_boxes[:,2]-new_boxes[:,0] #h
    new_boxes[:,6]=new_boxes[:,4]*new_boxes[:,5] #area
    return new_boxes  

In [ ]:
!pip3 install pickle5
import pickle5 as pickle

     |████████████████████████████████| 256 kB 12.8 MB/s 


In [12]:
with open('/content/drive/MyDrive/researchLab/MIMIC_CXR_TESTING_DATASET_IMAGE.pkl', 'rb') as f:
        test_image = pickle.load(f)
with open('/content/drive/MyDrive/researchLab/MIMIC_CXR_TESTING_DATASET_TEXT.pkl', 'rb') as f:
        test_text = pickle.load(f)
with open('/content/drive/MyDrive/researchLab/MIMIC_CXR_TRAINING_DATASET_IMAGE.pkl', 'rb') as f:
        train_image = pickle.load(f)
with open('/content/drive/MyDrive/researchLab/MIMIC_CXR_TRAINING_DATASET_TEXT.pkl', 'rb') as f:
        train_text = pickle.load(f)

In [13]:
training = PatientDataset(df = train_text, vf = train_image, model = args.model)
testing = PatientDataset(df = test_text, vf = test_image, model = args.model)

In [14]:
assert set(list(test_image.keys())) == set(test_text.study_id.values), "Visual Features are inconsistent with openI dataset"

In [15]:
train_loader = DataLoader(training, batch_size=args.batch_size,shuffle=True, num_workers=0,drop_last=True, pin_memory=True)
test_loader = DataLoader(testing, batch_size=128,shuffle=False, num_workers=0,drop_last=False, pin_memory=True)

In [16]:
findings = list(train_text.columns[5:-2])
findings

['Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Enlarged Cardiomediastinum',
 'Fracture',
 'Lung Lesion',
 'Lung Opacity',
 'No Finding',
 'Pleural Effusion',
 'Pleural Other',
 'Pneumonia',
 'Pneumothorax']

In [17]:
from vqa_model import VQAModel
#init model
model = VQAModel(num_answers = len(findings), model = args.model)

100%|██████████| 407873900/407873900 [00:16<00:00, 24496157.34B/s]


In [18]:
#load pretrained weights
model.encoder.load(args.load_pretrained)

Load VISUALBERT PreTrained Model from /content/Transformers-VQA/models/pretrained/visualbert.th

Weights in loaded but not in model:
cls.predictions.bias
cls.predictions.decoder.weight
cls.predictions.transform.LayerNorm.bias
cls.predictions.transform.LayerNorm.weight
cls.predictions.transform.dense.bias
cls.predictions.transform.dense.weight
cls.seq_relationship.bias
cls.seq_relationship.weight

Weights in model but not in loaded:



In [19]:
#send to GPU
model = model.cuda()

In [20]:
import torch
loss = torch.nn.BCEWithLogitsLoss()

In [21]:
from src.optimization import BertAdam
optim = BertAdam(list(model.parameters()),lr=args.lr,warmup=0.1,t_total=len(train_loader)*args.epochs)
# t_total denotes total training steps
# batch_per_epoch = len(train_loader)
# t_total = int(batch_per_epoch * args.epochs)

In [22]:
# Evaluation function, we will report the AUC and accuray of each finding
import tensorflow as tf
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
accuracy = tf.keras.metrics.Accuracy()
AUC = tf.keras.metrics.AUC()

def eval(target, pred):
    b = np.zeros_like(pred)
    b[np.arange(len(pred)), pred.argmax(1)] = 1
    pred = b
    acc_list = []
    precision_list_tf = []
    recall_list_tf = []
    accuracy_list_tf = []
    AUC_list_tf = []
    precision_list_ker = []
    recall_list_ker= []
    accuracy_list_ker = []
    #print('Averaged Recall: '+str(recall_score(target, pred, average='weighted')))
    #print('Averaged Precision: '+str(precision_score(target, pred ,average='weighted')))
    #print('Averaged Accuracy: '+str(accuracy_score(target, pred)))
    for i, d in enumerate(findings[:-1]):
        precision.update_state(target[:,i], (pred[:,i]>=0.5))
        accuracy.update_state(target[:,i], (pred[:,i]>=0.5))
        recall.update_state(target[:,i], (pred[:,i]>=0.5))
        AUC.update_state(target[:,i], (pred[:,i]>=0.5))
        precision_list_ker.append(precision_score(target, pred ,average='weighted'))
        accuracy_list_ker.append(accuracy_score(target, pred))
        recall_list_ker.append(recall_score(target, pred, average='weighted'))
        precision_list_tf.append(precision.result().numpy())
        accuracy_list_tf.append(recall.result().numpy())
        recall_list_tf.append(accuracy.result().numpy())
        AUC_list_tf.append(AUC.result().numpy())
        acc = np.mean(target[:,i] == (pred[:,i]>=0.5))
        #print(i,d,acc)
        acc_list.append(acc)
    print('Averaged AUC : '+str(np.average(acc_list)))
    print('Averaged AUC tf : '+str(np.average(AUC_list_tf)))
    print('Averaged Recall tf: '+str(np.average(recall_list_tf)))
    print('Averaged Precision tf: '+str(np.average(precision_list_tf)))
    print('Averaged Accuracy tf: '+str(np.average(accuracy_list_tf)))
    print('Averaged Recall sklearn: '+str(np.average(recall_list_ker)))
    print('Averaged Precision sklearn: '+str(np.average(precision_list_ker)))
    print('Averaged Accuracy sklearn: '+str(np.average(accuracy_list_ker)))

In [23]:
sgmd = torch.nn.Sigmoid()

In [24]:
from param import args
args.__dict__

{'batch_size': 18,
 'dropout': 0.1,
 'epochs': 2,
 'fast': False,
 'from_scratch': False,
 'load_pretrained': '/content/Transformers-VQA/models/pretrained/visualbert.th',
 'load_trained': None,
 'lr': 0.0001,
 'max_seq_length': 128,
 'mce_loss': False,
 'model': 'visualbert',
 'multiGPU': False,
 'num_workers': 0,
 'optim': 'bert',
 'optimizer': 'bert',
 'output': 'models/trained/',
 'seed': 9595,
 'test': None,
 'tiny': False,
 'tqdm': True,
 'train': 'train,nominival',
 'valid': 'minival'}

In [25]:
!pip install ipywidgets widgetsnbextension pandas-profiling

In [27]:
from tqdm.notebook import tqdm

iter_wrapper = (lambda x: tqdm(x, total=len(train_loader))) if args.tqdm else (lambda x: x)
best_valid = 0
for epoch in range(args.epochs):
  epoch_loss = 0
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(train_loader)):
    #print(cxr_id, feats, boxes, report, target)
    model.train()
    optim.zero_grad()
    feats, boxes, target = feats.cuda(), boxes.cuda(), target.cuda()
    logit = model(feats, boxes, report)
    running_loss = loss(logit, target)
    running_loss = running_loss * logit.size(1)
    epoch_loss += running_loss
    running_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
    optim.step()
  print("Epoch "+str(epoch)+": Training Loss: "+str(epoch_loss/len(train_loader)))
  print('Evaluation: ')
  model.eval()
  logit_list, target_list = [], []
  iter_wrapper = (lambda x: tqdm(x, total=len(test_loader)))
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(test_loader)):
    target_list.append(target)
    with torch.no_grad():
      feats, boxes = feats.cuda(), boxes.cuda()
      #print(feats, boxes)
      logit = model(feats, boxes, report)
      logit_list.append(sgmd(logit).cpu().numpy())
  eval(np.concatenate(target_list,axis = 0), np.concatenate(logit_list,axis = 0))

  0%|          | 0/833 [00:00<?, ?it/s]

/content/Transformers-VQA/src/optimization.py:142: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1050.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Epoch 0: Training Loss: tensor(3.1615, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/52 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

...

  0%|          | 0/52 [00:00<?, ?it/s]

Epoch 1: Training Loss: tensor(2.1844, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/52 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

Averaged AUC : 0.9220474613686535
Averaged AUC tf : 0.71837664
Averaged Recall tf: 0.90961796
Averaged Precision tf: 0.73643994
Averaged Accuracy tf: 0.46106783
Averaged Recall sklearn: 0.5088723526044648
Averaged Precision sklearn: 0.6806189978516543
Averaged Accuracy sklearn: 0.4607164358819988


In [28]:
final_list = []

for index, item in enumerate(target_list):
    for index_1, item in enumerate(target_list[index]):
        print(logit_list[index][index_1])
        joined = target_list[index][index_1] +  logit_list[index][index_1]
        set_1 = [target_list[index][index_1].numpy(),logit_list[index][index_1]]
        final_list.append(set_1)
        #final_list.append(target_list[index][index_1]+logit_list[index][index_1])
print(final_list)
df_final = pd.DataFrame(final_list, columns = ['actual', 'VBert_prediction'])
df_final.to_pickle('VBERT_actual_prediction_testing.pkl')

Streaming output truncated to the last 5000 lines.
       0.00857738, 0.13679574, 0.04994646], dtype=float32)], [array([0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32), array([7.1856816e-04, 4.5217136e-01, 5.6297351e-02, 7.0909762e-01,
       1.9041041e-02, 4.3894993e-03, 1.9589243e-02, 7.6192093e-01,
       1.1116486e-02, 2.3039332e-02, 1.1986551e-02, 1.8741405e-01,
       4.6568909e-03], dtype=float32)], [array([1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0.], dtype=float32), array([0.07695018, 0.4392416 , 0.15938888, 0.87925553, 0.01630786,
       0.005566  , 0.01017931, 0.02553642, 0.01307912, 0.9442659 ,
       0.00467436, 0.06971909, 0.02068898], dtype=float32)], [array([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.], dtype=float32), array([0.12174768, 0.19620703, 0.10232928, 0.03794569, 0.04444663,
       0.01372849, 0.05426369, 0.90868497, 0.00765644, 0.89085674,
       0.01610776, 0.16450591, 0.04019444], dtype=float32)], [array([0., 0., 0., 0., 0.